In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib agg


import global_settings
from utils import settings, experiments, results
import jax
from multiprocessing import Pool
from tqdm import tqdm
import jax.numpy as jnp
import os

In [2]:
experiment = experiments.ExperimentSampleConstrained(
    settings=settings.SettingsExperimentSample(
        output_path=global_settings.PATH_RESULTS,
        dataset="sinusoidal",
        dataset_normalization="standardization",
        hidden_layers=1,
        hidden_neurons=3,
        activation="tanh",
        activation_last_layer="none",
        num_warmup=2**10,
        statistic="reduced",
        statistic_p=0.95,
        samples_per_mode=64,
        identifiable_modes=4,
        pool_size=8,
        seed=0
    )
)

In [3]:
def run_parallel(key):
    experiment.sample(key)
    return experiment._mcmc.get_samples()

rng_key, *sub_keys = jax.random.split(jax.random.PRNGKey(experiment._settings.seed), experiment._sample_statistics["num_chains"] + 1)
with Pool(experiment._settings.pool_size) as p:
    samples_parallel = list(tqdm(p.imap(run_parallel, sub_keys), total=experiment._sample_statistics["num_chains"]))

samples = {}
for samples_run in samples_parallel:
    for key in samples_run.keys():
        if key not in samples:
            samples[key] = samples_run[key]
        else:
            samples[key] = jnp.concatenate([samples[key], samples_run[key]])
experiment._samples = samples

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [01:17<00:00,  3.32it/s]


In [4]:
print(f"saved as {experiment.save()}")

saved as /home/gw/data/experiments/master_thesis/results/bd5a1e9c17b4a149e37bef8d54e2d196.tar.gz
